In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder , FunctionTransformer
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.metrics import plot_confusion_matrix, recall_score,\
    accuracy_score, precision_score, f1_score


In [ ]:
df_desc = pd.read_csv("data/key.csv", index_col = 0)
df = pd.read_csv("data/data_clean.csv")
df = df.drop("Unnamed: 0",axis=1)

In [ ]:
df.dtypes

In [ ]:
df

In [ ]:
def int_to_object(df):
    df["region_code"] = df["region_code"].astype(object)
    df["district_code"] = df["district_code"].astype(object)
    df["public_meeting"] = df["public_meeting"].astype(str)
    df["permit"] = df["permit"].astype(str)
    df["construction_year_bins"] = df["construction_year_bins"].astype(str)
    return df.dtypes

In [ ]:
df

In [ ]:
int_to_object(df)

In [ ]:
X = df.drop(["status_group"], axis=1)
y = df['status_group']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=465615615)

In [ ]:
X_train.dtypes

In [ ]:
def grab_numeric(df):
    return df.select_dtypes(include=['int64'])

In [ ]:
GrabNumeric = FunctionTransformer(grab_numeric)

In [ ]:
pipe= Pipeline(steps=[("num", GrabNumeric),
                      ( "ss", StandardScaler())])

In [ ]:
pipe.fit(X_train)

In [ ]:
pipe.transform(X_train)

In [ ]:
j = -1
for i in X_train.columns:
    j+=1
    print("Index Number", j,i)

In [ ]:
# We'll throw these mini-pipelines into our ColumnTransformer.

subpipe_num = Pipeline(steps=[('num_impute', SimpleImputer(strategy="median")),
                           ('ss', StandardScaler())])
subpipe_cat = Pipeline(steps=[('cat_impute', SimpleImputer(strategy='constant')),
                             ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))])

In [ ]:
CT = ColumnTransformer(transformers=[('subpipe_num', subpipe_num, [1,7]),
                                         ('subpipe_cat', subpipe_cat, [0,2,3,4,5,6,8,9,10,11,12,13,14,15,16,17,18])])

In [ ]:
gbc_model_pipe = Pipeline([('ct', CT), ('gbc', GradientBoostingClassifier(random_state=42))])

In [ ]:
gbc_model_pipe.fit(X_train, y_train)

In [ ]:
gbc_model_pipe.score(X_train, y_train)

In [ ]:
y_hat = gbc_model_pipe.predict(X_test)
y_hat

In [ ]:
print(f"""
Our first model's accuracy on the test set is {round(accuracy_score(y_test, y_hat), 2)}. \n
Our first model's recall on the test set is {round(recall_score(y_test, y_hat,average= 'macro'), 2)} \n
Our first model's precision on the test set is {round(precision_score(y_test, y_hat,average= 'macro'), 2)} \n
Our first model's f1-score on the test is {round(f1_score(y_test, y_hat,average= 'macro'), 2)}.
""")

In [ ]:
plot_confusion_matrix(gbc_model_pipe, X_test, y_test);

In [ ]:
X_train

In [ ]:
X_train

In [ ]:
CT.fit(X_train)

In [ ]:
X_train

In [ ]:
param_grid = {'learning_rate': [0.01,0.001, 0.0001,0.00001 ,0.000001],
               'max_depth': [3,4,5,6,7,8,9,10],
               'min_samples_leaf': [5,6,7,8,9,10],
               'n_estimators':[100, 200,300,400,500,600,700,800,900,1000]}

In [ ]:
gbc_cv_pipe = Pipeline(steps=([("ct", CT), 
                                    ("gbc", GridSearchCV(estimator=GradientBoostingClassifier(),
                                 param_grid=param_grid,
                                 n_jobs=-1))]))

In [ ]:
gbc_cv_pipe.fit(X_train, y_train)

In [ ]:
gbc_model_pipe2 = Pipeline([('ct', CT), ('gbc', GradientBoostingClassifier(learning_rate=.01, n_estimators=1000, random_state=42))])

In [ ]:
gbc_model_pipe.fit(X_train, y_train)

In [ ]:
gbc_model_pipe.score(X_train, y_train)

In [ ]:
gbc_model_pipe2 = Pipeline([('ct', CT), ('gbc', GradientBoostingClassifier(learning_rate=.01, n_estimators=1000, random_state=42))])

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
param_grid = {'alpha': [0.01,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],}

In [ ]:
gbc_cv_pipe = Pipeline(steps=([("ct", CT), 
                                    ("gbc", GridSearchCV(estimator=CatBoostClassifier(),
                                 param_grid=param_grid,
                                 n_jobs=-1))]))

In [ ]:
from catboost import CatBoostClassifier
from sklearn import preprocessing

In [ ]:
CT1

In [ ]:
cat_boost_pipe = Pipeline([('ct1', CT), ('cat', CatBoostClassifier())])

In [ ]:
cat_boost_pipe.fit(X_train, y_train)

In [ ]:
cat_boost_pipe.score(X_train, y_train)

In [ ]:
cross_validate(cat_boost_pipe, X_train, y_train, return_train_score=True)

In [ ]:
cat_boost_pipe1 = Pipeline([('ct', CT), ('cat', CatBoostClassifier())])
cat_boost_pipe1.fit(X_train, y_train)

In [ ]:
cat_boost_pipe1.score(X_train, y_train)

In [ ]:
cross_validate(cat_boost_pipe1, X_train, y_train, return_train_score=True)

In [ ]:
y_hat1 = cat_boost_pipe1.predict(X_test)
y_hat1

In [ ]:
from sklearn.metrics import make_scorer

In [ ]:
plot_confusion_matrix(cat_boost_pipe1, X_test, y_test);

In [ ]:
print(f"""
Our first model's accuracy on the test set is {round(accuracy_score(y_test, y_hat1), 2)}. \n
Our first model's recall on the test set is {round(recall_score(y_test, y_hat1,average= 'macro'), 2)} \n
Our first model's precision on the test set is {round(precision_score(y_test, y_hat1,average= 'macro'), 2)} \n
Our first model's f1-score on the test is {round(f1_score(y_test, y_hat1,average= 'macro'), 2)}.
""")